# Complex body TFA Inversion

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA) data on a regular grid of a model with a complex geometry.

In [84]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

### Auxiliary functions

In [85]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

# Input

### Importing model parameters

In [86]:
model_dir = 'data/model.pickle'
data_dir = 'data/flightlines_grid_2100pts_100km2_-150m/data.pickle'
#data_dir = 'eqlayer/direction_inc65_dec-40_h190m/eqlayer_depth190m.pickle'

In [87]:
with open(model_dir) as w:
        model = pickle.load(w)

### Observation points and observed data

In [88]:
with open(data_dir) as w:
        data = pickle.load(w)

In [89]:
# observed data and observation points
dobs = data['tfa_upward_obs']
xp = data['grid'][0]
yp = data['grid'][1]
zp = data['grid'][2]
N = data['grid'][3]

In [90]:
shit = []
shit.append([1,2])

### Parameters of the initial model

In [91]:
M = 15 # number of vertices per prism
L = 8 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization
incs = model['inc']
decs = model['dec']
intensity = model['intensity']

# depth to the top, thickness and radius
z0 = model['z0']
dz = 600.
r = 500.

# total magnetization
props = {'magnetization': utils.ang2vec(
        intensity, incs, decs)}
rin = np.zeros(M) + r
m0 = np.hstack((rin, np.zeros(2)))
m0 = np.resize(m0, P - 1) 
m0 = np.hstack((m0, dz)) # inicial parameters vector
model0 = mfun.param2polyprism(m0, M, L, z0, props) # list of classes of prisms

varz = 100.
varint = 3.
varinc = 5.
vardec = 5.

param_list = mfun.varying_param(z0, varz, intensity,
                                 varint, incs, varinc, decs, vardec)

# main field
inc, dec = [-21.5, -18.7]

# predict data
d0 = polyprism.tf(xp, yp, zp, model0, inc, dec)

In [92]:
param_list[0]

[200.0, {'magnetization': array([ 3.21361447, -2.74468605,  9.06307787])}]

### Limits

In [93]:
# limits for parameters in meters
rmin = 100.
rmax = 2000.
x0min = -1500.
x0max = 1500.
y0min = -1500.
y0max = 1500.
dzmin = 200.
dzmax = 700.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [94]:
# variation for derivatives
deltax = 0.01*np.max(x0max)
deltay = 0.01*np.max(y0max)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [95]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [96]:
if data['grid'][4] == 'flightlines':
    if incs == model['inc'] and decs == model['dec']:
        mypath = data_dir[:-11]+'tfa_inversion/multiple_inc%d_dec%d_r%d_int%d_z0%d_dz%d_varz%d_varint%d_varinc%d_vardec%d' % (incs, decs, rin[0], np.linalg.norm(props['magnetization']), z0, dz, varz, varint, varinc, vardec)
        if not os.path.isdir(mypath):
           os.makedirs(mypath)

In [97]:
# output of inversion
inversion = dict()

### Regularization parameters

In [98]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 1000.00
dlamb = 10.      # step for Marquadt's parameter

a1 = 0.0005  # adjacent radial distances within each prism
a2 = 0.003   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 0.05     # vertically adjacent origins
a6 = 0.00001   # zero order Tikhonov on adjacent radial distances
a7 = 0.04     # zero order Tikhonov on thickness of each prism

In [99]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [100]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

In [101]:
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs

In [102]:
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['initial_estimate'] = model0
inversion['initial_data'] = d0
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = alpha
inversion['tol'] = tol
inversion['main_field'] = [-21.5, -18.7]
inversion['var'] = [varz, varint, varinc, vardec]

### Inversion

In [103]:
inversion_results = []
for p in param_list:
    d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
        xp, yp, zp, m0, M, L, delta,
        itmax, itmax_marq, lamb,
        dlamb, tol, mmin, mmax,
        m_out, dobs, inc, dec,
        p[1], alpha, p[0], dz
    )
    inversion_results.append([d_fit, m_est, model_est, phi_list, model_list, dobs - d_fit, res_list])

it:  0   it_marq:  0   lambda: 1e+03   misfit: 1.09690e+04
it:  1   it_marq:  0   lambda: 1e+02   misfit: 9.55749e+03
it:  2   it_marq:  0   lambda: 1e+01   misfit: 5.33827e+03
it:  3   it_marq:  0   lambda: 1e+00   misfit: 3.87493e+03
it:  4   it_marq:  0   lambda: 1e-01   misfit: 5.19915e+04
it:  4   it_marq:  1   lambda: 1e+00   misfit: 8.48288e+02
it:  5   it_marq:  0   lambda: 1e-01   misfit: 4.50228e+04
it:  5   it_marq:  1   lambda: 1e+00   misfit: 4.17376e+04
it:  5   it_marq:  2   lambda: 1e+01   misfit: 4.46894e+02
it:  6   it_marq:  0   lambda: 1e+00   misfit: 4.24326e+04
it:  6   it_marq:  1   lambda: 1e+01   misfit: 3.15133e+02
it:  7   it_marq:  0   lambda: 1e+00   misfit: 4.17124e+04
it:  7   it_marq:  1   lambda: 1e+01   misfit: 6.14044e+03
it:  7   it_marq:  2   lambda: 1e+02   misfit: 2.94085e+02
it:  8   it_marq:  0   lambda: 1e+01   misfit: 1.30705e+04
it:  8   it_marq:  1   lambda: 1e+02   misfit: 2.79193e+02
it:  9   it_marq:  0   lambda: 1e+01   misfit: 3.79411e+

# Results

In [104]:
inversion['results'] = inversion_results

In [105]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)

### 